<a href="https://colab.research.google.com/github/Yetibi/Autoreel/blob/main/LIMPIEZA_Y_PROCESAMIENTO_ARCHIVOS_MENSUALES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CODIGO LiMPIEZA Y TRANSFORMACION DE DATOS (BASADO EN PYTHON PARA EXCEL)

In [11]:
# Paso 1: Montar Google Drive en Colab
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Paso 2: Cargar el archivo desde Google Drive
import pandas as pd

# Ruta del archivo en Google Drive (ajusta la ruta y el nombre del archivo según sea necesario)
ruta_archivo = '/content/drive/My Drive/Data/nombre_del_archivo.xlsx'

# Cargar el archivo en un DataFrame de pandas
df = pd.read_excel(ruta_archivo)

# Paso 3: Código de limpieza
# Aquí está el código unificado que ya funcionó
import numpy as np

# Limpieza de nombres de columnas
df.columns = df.columns.str.strip()

# Lista de columnas clave
columnas_clave = [
    "SERVICIO", "CLIENTE", "CELULAR", "VEHICULO", "PLACA", "OPERARIO",
    "COMISION OPERARIO", "BANCOLOMBIA", "EFECTIVO", "DATAFONO", "TOTAL PAGADO"
]

# Limpiar espacios internos en columnas clave
df[columnas_clave] = df[columnas_clave].applymap(
    lambda x: str(x).strip() if isinstance(x, str) else x
)

# Eliminar filas vacías en "SERVICIO"
def fila_vacia_en_servicio(fila):
    if fila["CATEGORIA"].strip().upper() != "SERVICIOS":
        return False
    return all(
        pd.isna(fila[col]) or str(fila[col]).strip() in ["", "0", "None", "nan"]
        for col in columnas_clave
    )
df_tratada = df[~df.apply(fila_vacia_en_servicio, axis=1)].reset_index(drop=True)

# Eliminar filas que son encabezados repetidos en "SERVICIOS"
encabezados_servicio = [
    "# ORDEN", "SERVICIO", "CLIENTE", "CELULAR", "VEHICULO", "PLACA", "OPERARIO",
    "COMISION OPERARIO", "BANCOLOMBIA", "EFECTIVO", "DATAFONO", "TOTAL PAGADO"
]

def es_fila_encabezado(fila):
    if str(fila.get("CATEGORIA", "")).strip().upper() != "SERVICIOS":
        return False
    coincidencias = sum(
        str(fila.get(col, "")).strip().upper() == col.strip().upper()
        for col in encabezados_servicio
    )
    return coincidencias >= len(encabezados_servicio) - 1

df_tratada = df_tratada[~df_tratada.apply(es_fila_encabezado, axis=1)].reset_index(drop=True)

# Renombrar columnas
df_tratada = df_tratada.rename(columns={
    "SERVICIO": "CONCEPTO",
    "TOTAL PAGADO": "VALOR ($)"
})

# Crear columna "TIPO MOVIMIENTO"
categorias_egreso = [
    "COMISION", "GASTOS Ò PAGOS PROVEEDORES", "VALOR PAGOS TRANSFERENCIAS", "ENTREGA EFECTIVO DIA A:"
]
df_tratada["TIPO MOVIMIENTO"] = df_tratada["CATEGORIA"].apply(
    lambda x: "EGRESO" if str(x).strip().upper() in categorias_egreso else "INGRESO"
)

# Crear columna "FORMA DE PAGO"
def detectar_forma_pago(row):
    formas = []
    if pd.to_numeric(row.get("EFECTIVO", 0), errors="coerce") > 0:
        formas.append("EFECTIVO")
    if pd.to_numeric(row.get("BANCOLOMBIA", 0), errors="coerce") > 0:
        formas.append("BANCOLOMBIA")
    if pd.to_numeric(row.get("DATAFONO", 0), errors="coerce") > 0:
        formas.append("DATAFONO")
    return "/".join(formas) if formas else None

df_tratada["FORMA DE PAGO"] = df_tratada.apply(detectar_forma_pago, axis=1)

# Asegurar valores comparables
df_tratada["CATEGORIA"] = df_tratada["CATEGORIA"].astype(str).str.strip().str.upper()

# Asignación condicional de "EFECTIVO INGRESO" y "EFECTIVO EGRESO"
def asignar_efectivos(row):
    categoria = row["CATEGORIA"]
    valor = pd.to_numeric(row["VALOR ($)"], errors="coerce") or 0
    efectivo = pd.to_numeric(row["EFECTIVO"], errors="coerce") or 0

    if categoria in ["BASE DÍA", "INGRESO EFECTIVO"]:
        return pd.Series([valor, 0])
    elif categoria == "SERVICIOS" and efectivo > 0:
        return pd.Series([efectivo, 0])
    elif categoria in ["ADELANTOS O PRESTAMOS", "ENTREGA EFECTIVO DIA A", "GASTOS Ò PAGOS PROVEEDORES"]:
        return pd.Series([0, valor])
    else:
        return pd.Series([efectivo, None])

df_tratada[["EFECTIVO INGRESO", "EFECTIVO EGRESO"]] = df_tratada.apply(asignar_efectivos, axis=1)

# Clasificar movimientos y columnas finales
columnas_final = [
    "FECHA", "# ORDEN", "CONCEPTO", "CATEGORIA", "TIPO MOVIMIENTO", "CLIENTE", "CELULAR",
    "VEHICULO", "PLACA", "OPERARIO", "FORMA DE PAGO", "BANCOLOMBIA", "EFECTIVO", "DATAFONO",
    "INGRESO POR TRANSFERENCIA", "VALOR ($)", "BANCO VALOR PAGOS TRANSFERENCIAS", "BANCO INGRESO EFECTIVO",
    "EFECTIVO INGRESO", "EFECTIVO EGRESO"
]
df_tratada = df_tratada[[col for col in columnas_final if col in df_tratada.columns]]

# Paso 4: Guardar el archivo procesado en Google Drive
# Ruta para guardar el archivo procesado
ruta_guardado = '/content/drive/My Drive/Data/nombre_del_archivo_limpio.xlsx'

# Guardar el DataFrame procesado en formato Excel
df_tratada.to_excel(ruta_guardado, index=False)

print(f"Archivo limpio guardado en: {ruta_guardado}")

NameError: name 'xl' is not defined